<a href="https://colab.research.google.com/github/hby1117/class2025Spring/blob/main/06_NLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing 2

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import nltk
import pandas as pd
import string

## Word Count

In [ ]:
df = pd.read_csv("alice.csv")
tokenizer = RegexpTokenizer("[\w]+")
df["Tokens"] = df["Text"].str.lower().apply(tokenizer.tokenize)
df["WordCount"] = df["Tokens"].str.len()

### Number of words in each book

In [ ]:
total_result = df.groupby("Book")["WordCount"].sum().reset_index()
colors = ['#DC8458', '#950702']
total_result.plot(x="Book", y="WordCount", kind="bar", color=colors)
plt.xticks(ticks=[0, 1], labels=[1, 2])
plt.show()

### Average number of words per chapter of each book

In [ ]:
mean_result = df.groupby("Book")["WordCount"].mean().reset_index()
mean_result.plot(x="Book", y="WordCount", kind="bar", color=colors)
plt.xticks(ticks=[0, 1], labels=[1, 2])
plt.show()

### Shortest chapter of each book

In [ ]:
min_result = df.groupby("Book")["WordCount"].min().reset_index()
min_result.plot(x="Book", y="WordCount", kind="bar", color=colors)
plt.xticks(ticks=[0, 1], labels=[1, 2])
plt.show()

### Longest chapter of each book

In [ ]:
max_result = df.groupby("Book")["WordCount"].max().reset_index()
max_result.plot(x="Book", y="WordCount", kind="bar", color=colors)
plt.xticks(ticks=[0, 1], labels=[1, 2])
plt.show()

## Sentiment Analysis

In [ ]:
sent_df = pd.read_csv("alice_sent.csv")

def remove_punctuations(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, "")
    return text

  sent_df["Sentence"] = sent_df["Sentence"].apply(remove_punctuations)
  sent_df["Sentence"] = sent_df["Sentence"].apply(lambda x: x.lower())

In [ ]:
nltk.download('vader_lexicon')
analyzer = nltk.sentiment.vader.SentimentIntensityAnalyzer()
sent_df["Score"] = sent_df["Sentence"].apply(analyzer.polarity_scores)

In [ ]:
sent_df["TotalScore"] = sent_df["Score"].apply(lambda x: x["compound"])
sent_df["PosScore"] = sent_df["Score"].apply(lambda x: x["pos"])
sent_df["NeuScore"] = sent_df["Score"].apply(lambda x: x["neu"])
sent_df["NegScore"] = sent_df["Score"].apply(lambda x: x["neg"])

In [ ]:
sent_df = sent_df.drop(["Score"], axis=1)
sent_df["PosFlag"] = sent_df["TotalScore"].apply(lambda x: x >= 0.05)
sent_df["NeuFlag"] = sent_df["TotalScore"].apply(lambda x: x > -0.05 and x < 0.05)
sent_df["NegFlag"] = sent_df["TotalScore"].apply(lambda x: x <= -0.05)

In [ ]:
sent_df.groupby("Book")["TotalScore"].mean()

In [ ]:
num_pos = sent_df['PosFlag'].sum()
num_neg = sent_df['NegFlag'].sum()
num_neu = sent_df['NeuFlag'].sum()
print(f"There are {num_pos} positive sentences.")
print(f"There are {num_neg} negative sentences.")
print(f"There are {num_neu} neutral sentences.")

In [ ]:
sent_df.groupby(["Chapter", "Book"])["TotalScore"].mean().unstack().plot(subplots=True)
plt.show()